In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as sc
from matplotlib.animation import FFMpegWriter, FuncAnimation
from fplanck import fokker_planck, boundary, gaussian_pdf
from mpl_toolkits.mplot3d import Axes3D

In [2]:
dims = 3
s = 10
r = 28
b = 8/3
sigma = np.ones(dims) * 4/5

In [3]:
T = 0.5 * np.square(sigma)/sc.k
drag = np.ones(dims)

In [4]:
def force(x1, x2, x3):
    nu1 = -s * (x1 - x2)
    nu2 = r * x1 - x2 - x1 * x3
    nu3 = x1 * x2 - b * x3
    return np.stack([nu1, nu2, nu3])

In [5]:
extent = np.ones(dims) * 5
resolution = np.ones(dims) * 0.1
sim = fokker_planck(temperature=T, drag=drag, extent=extent,
            resolution=resolution, force=force)

In [6]:
### time-evolved solution
pdf = gaussian_pdf(center=(1.5, -1.5, 1.5), width=1)
p0 = pdf(*sim.grid)

Nsteps = 100
time, Pt = sim.propagate_interval(pdf, 1e-2, Nsteps=Nsteps)

In [7]:
import matplotlib.colors
cmap = plt.cm.viridis

In [8]:
import sys

fig, ax = plt.subplots(subplot_kw=dict(projection='3d'), constrained_layout=True)

x = sim.grid[0].ravel()
y = sim.grid[1].ravel()
z = sim.grid[2].ravel()
p0 = p0.ravel()

norm = matplotlib.colors.Normalize(vmin=p0.min(), vmax=p0.max())
scat = ax.scatter(x, y, z, color=cmap(norm(p0)), alpha=0.1)

# ax.set_zlim([0,np.max(Pt)/3])
ax.autoscale(False)

def update(i):
    global scat
    scat.remove()
    scat = ax.scatter(x, y, z, color=cmap(norm(Pt[i].ravel())), alpha=0.1)
    return [scat]

anim = FuncAnimation(fig, update, frames=range(Nsteps), interval=10)
ax.set(xlabel='x1', ylabel='x2', zlabel='x3')

# # saving to m4 using ffmpeg writer
writervideo = FFMpegWriter(fps=60)
anim.save('/project/fokkerplanckgauss.mp4', writer=writervideo)

plt.close()

0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
